# 📊 Preenchimento automático de CST e CFOP com base no NCM

Faça upload das planilhas `ListaProdutos.xlsx` e `Base.xlsx`. O notebook irá preencher automaticamente os campos de CST Saída e CFOP Venda com base no NCM.

In [ ]:
import pandas as pd
from IPython.display import FileLink
import io

# Upload de arquivos
from google.colab import files

print("📁 Faça upload da planilha ListaProdutos.xlsx")
uploaded1 = files.upload()
lista_file = next(iter(uploaded1.values()))

print("📁 Faça upload da planilha Base.xlsx")
uploaded2 = files.upload()
base_file = next(iter(uploaded2.values()))


In [ ]:
# Ler os arquivos
lista_df = pd.read_excel(io.BytesIO(lista_file), dtype={'NCM': str})
base_df = pd.read_excel(io.BytesIO(base_file), dtype={'NCM': str})

# Corrigir e normalizar NCMs
lista_df['NCM'] = lista_df['NCM'].astype(str).str.replace('.0', '', regex=False).str.zfill(8)
base_df['NCM'] = base_df['NCM'].astype(str).str.replace('.0', '', regex=False).str.zfill(8)

# Fazer merge com base no NCM
merged_df = pd.merge(lista_df, base_df[['NCM', 'CST Saida', 'CFOP Venda']], on='NCM', how='left')

# Substituir os campos originais se existirem
merged_df['CST Saida'] = merged_df['CST Saida_y'].combine_first(merged_df.get('CST Saida_x'))
merged_df['CFOP Venda'] = merged_df['CFOP Venda_y'].combine_first(merged_df.get('CFOP Venda_x'))

# Limpar colunas duplicadas
merged_df = merged_df.drop(columns=[col for col in merged_df.columns if col.endswith('_x') or col.endswith('_y')])


In [ ]:
# Salvar o arquivo final
output_filename = "ListaProdutos_Preenchida.xlsx"
merged_df.to_excel(output_filename, index=False)

print("✅ Arquivo preenchido com sucesso!")
FileLink(output_filename)
